# Single-Session vmPFC θ SFC & Hipp γ↔vmPFC θ ir-PAC
Working spec for a one-session pipeline that quantifies hippocampal spike-to-vmPFC theta phase locking and hippocampal gamma amplitude coupling to vmPFC theta phase.

## 1. Setup & Imports

In [1]:
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid", context="talk")
warnings.filterwarnings("ignore", category=RuntimeWarning)

from nwb_analysis import get_subject_files
from nwb_analysis.cfc import (
    prepare_session_structures,
    compute_vmPFC_theta_phase,
    compute_hipp_gamma_envelope,
    compute_irpac,
    run_session_stats,
    plot_session_summary,
    save_session_outputs,
)


## 2. Config & Session Selection

In [2]:
# Session + storage config
DATA_DIR = Path('/Users/jundazhu/SBCAT/000673')
SESSION_ID = 'SBCAT_Example_001'  # can be partial stem or left as example
SESSION_PATH_OVERRIDE = None      # Path('/absolute/path/to/sub-5_ses-1_ecephys+image.nwb') to skip globbing
OUT_DIR = Path('outputs')
OUT_DIR.mkdir(parents=True, exist_ok=True)
USE_ONLY_CORRECT = True

SESSION_OUT_DIR = OUT_DIR / SESSION_ID / 'sfc_irpac_single_session'
SESSION_OUT_DIR.mkdir(parents=True, exist_ok=True)

CONDITIONS = ['L1', 'L3']
EPOCH_ANALYZE = (-0.5, 2.0)       # s window used for analyses relative to maintenance onset
EPOCH_LABEL = 'encoding1'
THETA_BAND = (3.0, 7.0)
PHASE_BAND_LABEL = 'theta'
GAMMA_BAND = (30.0, 140.0)
GAMMA_BAND_LABEL = 'gamma'
CAR_MODE = 'hemisphere'
MIN_TRIALS_PER_COND = 20
PAC_SURR_N = 500
LAG_GRID = np.arange(-0.150, 0.155, 0.005)  # seconds
EXCLUDE_LAG_S = (0.010, 0.020)              # ignore central lags when reporting lag-swept PAC
PEAK_TO_PEAK_THRESH = 3000.0  # µV; set to None to skip automatic rejection
PAC_ALPHA = 0.05

ANALYSIS_PARAMS = {
    'conditions': CONDITIONS,
    'epoch_analyze': EPOCH_ANALYZE,
    'epoch_label': EPOCH_LABEL,
    'theta_band': THETA_BAND,
    'phase_band_label': PHASE_BAND_LABEL,
    'gamma_band': GAMMA_BAND,
    'gamma_band_label': GAMMA_BAND_LABEL,
    'car_mode': CAR_MODE,
    'min_trials': MIN_TRIALS_PER_COND,
    'pac_surrogates': PAC_SURR_N,
    'lag_grid_s': LAG_GRID.tolist(),
    'exclude_lag_s': EXCLUDE_LAG_S,
    'max_peak_to_peak': PEAK_TO_PEAK_THRESH,
    'pac_alpha': PAC_ALPHA,
}

print(f"Session ID: {SESSION_ID}")
print(f"Outputs → {SESSION_OUT_DIR}")


Session ID: SBCAT_Example_001
Outputs → outputs/SBCAT_Example_001/sfc_irpac_single_session


### Resolve session file

In [3]:
from pathlib import Path


def resolve_session_path(session_id: str, data_dir: Path, path_override: Path | None = None) -> Path:
    '''Resolve an NWB file path using override, direct path, or globbing.'''
    candidates = []
    if path_override is not None:
        candidates.append(Path(path_override).expanduser())
    if session_id:
        candidates.append(Path(session_id).expanduser())
        candidates.append((data_dir / session_id).expanduser())
    for candidate in candidates:
        if candidate.suffix == '.nwb' and candidate.exists():
            return candidate
    if session_id:
        matches = sorted(data_dir.rglob(f"*{session_id}*.nwb"))
        if matches:
            return matches[0]
    files = get_subject_files(data_dir)
    if not files:
        raise FileNotFoundError(f"No NWB files found under {data_dir}")
    if session_id:
        print(f"[warn] Could not find '{session_id}' under {data_dir}. Falling back to {files[0].name}")
    return files[0]


SESSION_PATH = resolve_session_path(SESSION_ID, DATA_DIR, SESSION_PATH_OVERRIDE)
print(f"Using NWB file: {SESSION_PATH}")


[warn] Could not find 'SBCAT_Example_001' under /Users/jundazhu/SBCAT/000673. Falling back to sub-1_ses-1_ecephys+image.nwb
Using NWB file: /Users/jundazhu/SBCAT/000673/sub-1/sub-1_ses-1_ecephys+image.nwb


## 3. Load Data via Adapter

In [4]:
session_data = prepare_session_structures(
    session_id=SESSION_ID,
    session_path=SESSION_PATH,
    conditions=CONDITIONS,
    use_only_correct=USE_ONLY_CORRECT,
    epoch_label=EPOCH_LABEL,
)

trial_table = session_data['trial_table']
spike_times_by_unit = session_data['spike_times_by_unit']
unit_meta = session_data['unit_meta']
lfp_by_channel = session_data['lfp_by_channel']
lfp_fs = session_data['lfp_fs']
lfp_start_time = session_data['lfp_start_time']
channel_meta = session_data['channel_meta']
session_meta = session_data['session_meta']

print(f"Trials: {len(trial_table)} | Units: {len(spike_times_by_unit)} | Channels: {len(lfp_by_channel)} @ {lfp_fs:.1f} Hz")
trial_table.head()


/Users/jundazhu/.pyenv/versions/3.10.17/lib/python3.10/site-packages/pynwb/ecephys.py:158: UserWarning: ElectricalSeries 'LFPs': The second dimension of data does not match the length of electrodes. Your data may be transposed.
  warnings.warn("%s '%s': The second dimension of data does not match the length of electrodes. "


Trials: 136 | Units: 46 | Channels: 70 @ 400.0 Hz


,condition_label,rt,epoch_onset_time,maint_onset_time,is_correct,id,loads,PicIDs_Encoding1,PicIDs_Encoding2,PicIDs_Encoding3,...,timestamps_Encoding1_end,timestamps_Encoding2,timestamps_Encoding2_end,timestamps_Encoding3,timestamps_Encoding3_end,timestamps_Maintenance,timestamps_Probe,timestamps_Response,response_accuracy,probe_in_out
0,L3,1.660094,2.193249,8.542684,True,0,3,201,101,501,...,4.203154,4.393779,6.401091,6.535497,8.542684,8.542684,11.746151,13.406245,1,1
1,L3,0.997281,14.870369,21.168179,True,1,3,202,102,502,...,16.906399,16.947618,18.955961,19.159867,21.168179,21.168179,24.032771,25.030052,1,1
2,L3,0.819468,26.257614,32.644268,True,2,3,301,401,103,...,28.265895,28.458301,30.466706,30.635925,32.644268,32.644268,35.436236,36.255704,1,1
3,L1,0.848031,37.644922,39.653265,True,3,1,503,0,0,...,39.653265,0.000000,0.000000,0.000000,0.000000,39.653265,42.396920,43.244951,1,1
4,L3,2.271156,44.583513,50.802136,True,4,3,402,203,504,...,46.591325,46.634856,48.643137,48.793824,50.802136,50.802136,53.367728,55.638884,1,0


## 4. Preprocessing (CAR, bandpass, Hilbert)

In [5]:
theta_phase = compute_vmPFC_theta_phase(
    lfp_by_channel=lfp_by_channel,
    channel_meta=channel_meta,
    trial_table=trial_table,
    sampling_rate=lfp_fs,
    lfp_start_time=lfp_start_time,
    epoch_analyze=EPOCH_ANALYZE,
    theta_band=THETA_BAND,
    car_mode=CAR_MODE,
    trial_pad_sec=0.5,
    filter_order=4,
)

hipp_gamma_amp = compute_hipp_gamma_envelope(
    lfp_by_channel=lfp_by_channel,
    channel_meta=channel_meta,
    trial_table=trial_table,
    sampling_rate=lfp_fs,
    lfp_start_time=lfp_start_time,
    epoch_analyze=EPOCH_ANALYZE,
    gamma_band=GAMMA_BAND,
    trial_pad_sec=0.5,
    filter_order=4,
)

print(f"vmPFC θ channels: {len(theta_phase)}")
print(f"Hipp γ channels: {len(hipp_gamma_amp)}")

vmPFC θ channels: 14
Hipp γ channels: 14


## 5. LFP Pair Formation (hipp γ channel × vmPFC θ channel)
Pairs enumerate hippocampal LFP channels supplying γ envelopes and vmPFC channels supplying θ phase. Channels must share a hemisphere when labels are available; otherwise they are paired globally.


In [6]:
hipp_channels = channel_meta[channel_meta['area'].str.contains('hipp', case=False, na=False)].copy()
vm_channels = channel_meta[channel_meta['area'].str.contains('vmpfc', case=False, na=False)].copy()
hipp_channels['hemisphere'] = hipp_channels.get('hemisphere', 'unknown').fillna('unknown')
vm_channels['hemisphere'] = vm_channels.get('hemisphere', 'unknown').fillna('unknown')

pairs = []
for hipp in hipp_channels.itertuples():
    for vm in vm_channels.itertuples():
        if hipp.hemisphere != 'unknown' and vm.hemisphere != 'unknown' and hipp.hemisphere != vm.hemisphere:
            continue
        pair_id = f"hipp{hipp.chan_id}_vm{vm.chan_id}"
        pairs.append({
            'pair_id': pair_id,
            'unit_id': hipp.chan_id,
            'hipp_chan_id': hipp.chan_id,
            'unit_channel': hipp.chan_id,
            'hipp_hemisphere': hipp.hemisphere,
            'chan_id': vm.chan_id,
            'vm_hemisphere': vm.hemisphere,
        })

pair_table = pd.DataFrame(pairs)
print(f"Candidate hipp×vmPFC channel pairs: {len(pair_table)}")
pair_table.head()


Candidate hipp×vmPFC channel pairs: 98


,pair_id,unit_id,hipp_chan_id,unit_channel,hipp_hemisphere,chan_id,vm_hemisphere
0,hipp22_vm67,22,22,22,L,67,L
1,hipp22_vm68,22,22,22,L,68,L
2,hipp22_vm69,22,22,22,L,69,L
3,hipp22_vm70,22,22,22,L,70,L
4,hipp22_vm71,22,22,22,L,71,L


## 7. Hipp γ ↔ vmPFC θ ir-PAC

In [7]:
pac_results = compute_irpac(
    pair_table=pair_table[:],
    trial_table=trial_table,
    theta_phase=theta_phase,
    gamma_envelope=hipp_gamma_amp,
    lfp_fs=lfp_fs,
    epoch_analyze=EPOCH_ANALYZE,
    conditions=CONDITIONS,
    min_trials=MIN_TRIALS_PER_COND,
    n_surrogates=PAC_SURR_N,
    lag_grid_s=None,
    exclude_lag_s=EXCLUDE_LAG_S,
    significance_alpha=PAC_ALPHA,
    phase_band=THETA_BAND,
    amp_band=GAMMA_BAND,
    phase_band_label=PHASE_BAND_LABEL,
    amp_band_label=GAMMA_BAND_LABEL,
    epoch_label=EPOCH_LABEL,
)

pac_results.head()

,pair_id,unit_id,hipp_chan_id,vm_chan_id,condition,phase_band_label,phase_band_lo_hz,phase_band_hi_hz,amp_band_label,amp_band_lo_hz,...,epoch_label,epoch_start_s,epoch_end_s,n_trials,min_trial_count,raw_pac,z_pac_theta_gamma,p_pac_theta_gamma,is_pac_significant,alpha
0,hipp22_vm67,22,22,67,L1,theta,3.0,7.0,gamma,30.0,...,encoding1,-0.5,2.0,70,66,0.000036,-0.086706,0.463074,False,0.05
1,hipp22_vm67,22,22,67,L3,theta,3.0,7.0,gamma,30.0,...,encoding1,-0.5,2.0,66,66,0.000061,0.166409,0.385230,False,0.05
2,hipp22_vm68,22,22,68,L1,theta,3.0,7.0,gamma,30.0,...,encoding1,-0.5,2.0,70,66,0.000028,-0.720452,0.746507,False,0.05
3,hipp22_vm68,22,22,68,L3,theta,3.0,7.0,gamma,30.0,...,encoding1,-0.5,2.0,66,66,0.000046,-0.353597,0.604790,False,0.05
4,hipp22_vm69,22,22,69,L1,theta,3.0,7.0,gamma,30.0,...,encoding1,-0.5,2.0,70,66,0.000040,0.060047,0.413174,False,0.05


## 8. Session-Level Stats

In [8]:
stats_summary = run_session_stats(
    sfc_results=None,
    pac_results=pac_results,
    conditions=CONDITIONS,
)

stats_summary


{'sfc_theta': None,
 'pac_theta_gamma': {'n_pairs': 98,
  'median_a': -0.46675939347082607,
  'median_b': 0.05916533523550037,
  'median_delta': 0.6427234485330999,
  'iqr_a': (-0.902063179704554, 0.05558644198551187),
  'iqr_b': (-0.29234034427188527, 0.7823214370394134),
  'p_value': 8.038477494581487e-07,
  'effect_size_r': -0.4984527984965512,
  'delta_ci95': (0.37268922891023676, 0.8781255460419031),
  'wilcoxon_stat': 1033.0,
  'wilcoxon_z': -4.9344309548565235}}

## 9. QC & Plots

In [9]:
example_pair_id = None
summary_figs = plot_session_summary(
    sfc_results=None,
    pac_results=pac_results,
    stats_summary=stats_summary,
    conditions=CONDITIONS,
    output_dir=SESSION_OUT_DIR,
    dpi=150,
)
print(f"Session summary figs: {summary_figs}")


Session summary figs: [PosixPath('outputs/SBCAT_Example_001/sfc_irpac_single_session/pac_session_summary.png')]


## 10. Save Artifacts

In [ ]:
artifacts = save_session_outputs(
    session_id=SESSION_ID,
    session_path=SESSION_PATH,
    output_dir=SESSION_OUT_DIR,
    pair_table=None,
    sfc_results=None,
    pac_results=pac_results,
    stats_summary=stats_summary,
    analysis_params=ANALYSIS_PARAMS,
)

artifacts


## 11. Appendix (runtime info)

In [ ]:
import json
import platform
import sys
from datetime import datetime

runtime_info = {
    'timestamp': datetime.utcnow().isoformat() + 'Z',
    'python': sys.version,
    'platform': platform.platform(),
    'session_meta': session_meta,
    'analysis_params': ANALYSIS_PARAMS,
}

print(json.dumps(runtime_info, indent=2))